In [378]:
import pandas as pd

In [379]:
def split_kolicina(kolicina):
    if pd.isna(kolicina) or kolicina == '':
        return pd.NA, pd.NA
    
    kolicina_str = str(kolicina).strip()
    parts = kolicina_str.rsplit(' ', 1)
    
    if len(parts) == 2:
        return parts[0], parts[1]
    else:
        return kolicina_str, pd.NA

In [380]:
# Function to convert Cyrillic to Latin script
def cyr_to_lat(text):
    if not isinstance(text, str):
        return text
        
    # Create mapping dictionary from Cyrillic to Latin
    cyr_to_lat_dict = {
        'А': 'A', 'Б': 'B', 'В': 'V', 'Г': 'G', 'Д': 'D', 'Ђ': 'Đ', 'Е': 'E', 'Ж': 'Ž',
        'З': 'Z', 'И': 'I', 'Ј': 'J', 'К': 'K', 'Л': 'L', 'Љ': 'Lj', 'М': 'M', 'Н': 'N',
        'Њ': 'Nj', 'О': 'O', 'П': 'P', 'Р': 'R', 'С': 'S', 'Т': 'T', 'Ћ': 'Ć', 'У': 'U',
        'Ф': 'F', 'Х': 'H', 'Ц': 'C', 'Ч': 'Č', 'Џ': 'Dž', 'Ш': 'Š',
        
        'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'ђ': 'đ', 'е': 'e', 'ж': 'ž',
        'з': 'z', 'и': 'i', 'ј': 'j', 'к': 'k', 'л': 'l', 'љ': 'lj', 'м': 'm', 'н': 'n',
        'њ': 'nj', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'ћ': 'ć', 'у': 'u',
        'ф': 'f', 'х': 'h', 'ц': 'c', 'ч': 'č', 'џ': 'dž', 'ш': 'š'
    }
    
    result = ""
    for char in text:
        result += cyr_to_lat_dict.get(char, char)
    
    return result

In [381]:
df = pd.read_csv('data.csv')
df.head()

,ID leka,Назив лека:,Генерички назив:,Облик и паковање:,Kolicina,Oblik,Doza,Врста решења:,АТЦ шифра:,JKL:,...,- земља:,Носилац дозволе:,Режим издавања:,Број решења:,Датум решења:,Датум истицања решења:,Сажетак карактеристика о леку:,Упутство за лек:,Текст за спољње и унутрашње паковање:,Одобрене измене:
0,382160,Rebif®,interferon beta-1a,rastvor za injekciju u ulošku; 22mcg/0.5mL; ul...,6 mL,rastvor za injekciju u ulošku,22mcg/0.5mL,ОБНОВА,L03AB07,328390.0,...,Italija,MERCK D.O.O. BEOGRAD,СЗР - Лек се може употребљавати у стационарној...,515-01-00501-17-006,07.08.2019.,07.08.2069.,pdf/382160/sazetak_karakteristika_lijeka.pdf,pdf/382160/upustvo_za_upotrebu.pdf,pdf/382160/tekst_za_spoljno_pakovanje.pdf,У припреми
1,382161,Rebif®,interferon beta-1a,rastvor za injekciju u ulošku; 22mcg/0.5mL; ul...,6 mL,rastvor za injekciju u ulošku,22mcg/0.5mL,ОБНОВА,L03AB07,328390.0,...,Švajcarska,MERCK D.O.O. BEOGRAD,СЗР - Лек се може употребљавати у стационарној...,515-01-00501-17-006,07.08.2019.,07.08.2069.,pdf/382161/sazetak_karakteristika_lijeka.pdf,pdf/382161/upustvo_za_upotrebu.pdf,pdf/382161/tekst_za_spoljno_pakovanje.pdf,У припреми
2,382175,Rebif®,interferon beta-1a,rastvor za injekciju u ulošku; 44mcg/0.5mL; ul...,6 mL,rastvor za injekciju u ulošku,44mcg/0.5mL,ОБНОВА,L03AB07,328389.0,...,Italija,MERCK D.O.O. BEOGRAD,СЗР - Лек се може употребљавати у стационарној...,515-01-00517-17-006,07.08.2019.,07.08.2069.,pdf/382175/sazetak_karakteristika_lijeka.pdf,pdf/382175/upustvo_za_upotrebu.pdf,pdf/382175/tekst_za_spoljno_pakovanje.pdf,У припреми
3,382176,Rebif®,interferon beta-1a,rastvor za injekciju u ulošku; 44mcg/0.5mL; ul...,6 mL,rastvor za injekciju u ulošku,44mcg/0.5mL,ОБНОВА,L03AB07,328389.0,...,Švajcarska,MERCK D.O.O. BEOGRAD,СЗР - Лек се може употребљавати у стационарној...,515-01-00517-17-006,07.08.2019.,07.08.2069.,pdf/382176/sazetak_karakteristika_lijeka.pdf,pdf/382176/upustvo_za_upotrebu.pdf,pdf/382176/tekst_za_spoljno_pakovanje.pdf,У припреми
4,390776,Crinone®,progesteron,"vaginalni gel; 8%; aplikator, 15x1.125g",16.875 g,vaginalni gel,0.08,ОБНОВА,G03DA04,4137040.0,...,Velika Britanija,MERCK D.O.O. BEOGRAD,Р - Лек се може издавати само уз лекарски рецепт,515-01-02136-17-004,30.07.2019.,30.07.2069.,pdf/390776/sazetak_karakteristika_lijeka.pdf,pdf/390776/upustvo_za_upotrebu.pdf,pdf/390776/tekst_za_spoljno_pakovanje.pdf,У припреми


In [382]:
df = df[['ID leka', 'Назив лека:', 'Kolicina', 'Oblik', 'Doza', 'EAN:', 'Произвођач:']]
df.columns = ['ID', 'Naziv', 'Kolicina', 'Oblik', 'Doza', 'EAN', 'Proizvodjac']
len(df)

6183

In [383]:
df = df[~df['Doza'].str.contains(r'[/+\\-]', na=False)]
len(df)

3392

In [384]:


df['Doza_jedinica_mjere'] = None
df['Snaga'], df['Doza_jedinica_mjere'] = zip(*df['Kolicina'].apply(split_kolicina))
df[['Snaga', 'Doza_jedinica_mjere']].head()

,Snaga,Doza_jedinica_mjere
4,16.875,g
7,30,kom
8,30,kom
13,12,kom
14,12,kom


In [385]:
# Split Doza into snaga (numeric value) and jedinica_mjere (unit)
import re

def split_doza(doza):
    if pd.isna(doza) or doza == '':
        return pd.NA, pd.NA
    
    doza_str = str(doza).strip()
    
    # Try to convert the entire string to float to check if it's just a number
    try:
        float_value = float(doza_str)
        return doza_str, pd.NA  # If it's just a number, no unit
    except ValueError:
        # Pattern to match numbers at the beginning followed by non-numeric characters
        pattern = r'^(\d+[.,]?\d*)(.+)$'
        match = re.match(pattern, doza_str)
        
        if match:
            # Extract numeric part and unit part
            snaga = match.group(1)
            jedinica_mjere = match.group(2).strip()
            return snaga, jedinica_mjere
        else:
            # If no digits at the beginning, check if there's a space
            parts = doza_str.rsplit(' ', 1)
            if len(parts) == 2:
                return parts[0], parts[1]
            else:
                return doza_str, pd.NA

df['Doza_snaga'] = None
df['Doza_jedinica_mjere'] = None
df['Doza_snaga'], df['Doza_jedinica_mjere'] = zip(*df['Doza'].apply(split_doza))

# Display results to verify
print(df[['Doza', 'Doza_snaga', 'Doza_jedinica_mjere']].head(10))

      Doza Doza_snaga Doza_jedinica_mjere
4     0.08       0.08                <NA>
7   0.25mg       0.25                  mg
8      1mg          1                  mg
13   400mg        400                  mg
14   400mg        400                  mg
17    10mg         10                  mg
18    10mg         10                  mg
19    20mg         20                  mg
20    20mg         20                  mg
23   1.5mg        1.5                  mg


In [386]:
# print all unique values in Doza_jedinica_mjere
print(df['Doza_jedinica_mjere'].unique())

[<NA> 'mg' 'mcg' 'g' 'LD50jed' 'i.j.' 'Mi.j.' 'Allergan j.' 'j.' 'mL'
 'x 10exp9 CFU' 'LSU' 'x 10exp7 CFU']


In [387]:
# remove all with [<NA> 'LD50jed' 'x 10exp9 CFU' 'LSU' 'x 10exp7 CFU
df = df[~df['Doza_jedinica_mjere'].isin([pd.NA, 'LD50jed', 'x 10exp9 CFU', 'LSU', 'x 10exp7 CFU'])]

In [388]:
# print all unique values in Doza_jedinica_mjere
print(df['Doza_jedinica_mjere'].unique())

['mg' 'mcg' 'g' 'i.j.' 'Mi.j.' 'Allergan j.' 'j.' 'mL']


In [389]:
# make a sepearate file for doze.csv

df_doze = df[['ID', 'Doza_snaga', 'Doza_jedinica_mjere']]

df_doze.to_csv('doze.csv', index=False, encoding='utf-8-sig')

In [390]:
df.head()

,ID,Naziv,Kolicina,Oblik,Doza,EAN,Proizvodjac,Doza_jedinica_mjere,Snaga,Doza_snaga
7,394112,Mirapexin®,30 kom,tableta,0.25mg,8.606100e+12,BOEHRINGER INGELHEIM PHARMA GMBH & CO.KG,mg,30,0.25
8,394124,Mirapexin®,30 kom,tableta,1mg,8.606100e+12,BOEHRINGER INGELHEIM PHARMA GMBH & CO.KG,mg,30,1
13,395653,Brufen® Vivo,12 kom,film tableta,400mg,5.099150e+12,MCDERMOTT LABORATORIES LIMITED T/A GERARD LABO...,mg,12,400
14,395655,Brufen® Vivo,12 kom,film tableta,400mg,5.099150e+12,MYLAN HUNGARY KFT.,mg,12,400
17,395876,Tadalafil Mylan,4 kom,film tableta,10mg,5.099150e+12,MYLAN HUNGARY KFT.,mg,4,10


In [391]:
df_unique_jedinice = df[['Doza_jedinica_mjere']].drop_duplicates()
df_unique_jedinice = df_unique_jedinice[df_unique_jedinice['Doza_jedinica_mjere'].notna()]
df_unique_jedinice = df_unique_jedinice[df_unique_jedinice['Doza_jedinica_mjere'] != '']

df_unique_jedinice = df_unique_jedinice.rename(columns={'Doza_jedinica_mjere': 'Doza_jedinica_mjere'})

df_unique_jedinice = df_unique_jedinice.sort_values(by='Doza_jedinica_mjere')

df_unique_jedinice = df_unique_jedinice.reset_index(drop=True)
df_unique_jedinice.index = df_unique_jedinice.index + 1

df_unique_jedinice.to_csv('jedinice_mjere.csv', index=True, header=True, index_label='Index')

In [392]:
df_unique_vrste = df[['Oblik']].drop_duplicates()
df_unique_vrste = df_unique_vrste[df_unique_vrste['Oblik'].notna()]
df_unique_vrste = df_unique_vrste[df_unique_vrste['Oblik'] != '']

df_unique_vrste['Oblik'] = df_unique_vrste['Oblik'].apply(cyr_to_lat)

df_unique_vrste['Oblik'] = df_unique_vrste['Oblik'].str.capitalize()

df_unique_vrste = df_unique_vrste.sort_values(by='Oblik')

df_unique_vrste = df_unique_vrste.reset_index(drop=True)
df_unique_vrste.index = df_unique_vrste.index + 1

df_unique_vrste.to_csv('oblici.csv', index=True, header=True, index_label='Index')

In [393]:
df_barkodovi = df[['ID', 'EAN', 'Naziv', 'Doza_snaga', 'Doza_jedinica_mjere', 'Oblik']].copy()

# Convert EAN to string without decimal point
df_barkodovi['EAN'] = df_barkodovi['EAN'].apply(
    lambda x: str(int(float(x))) if pd.notna(x) else ''
)

# Convert Oblik from Cyrillic to Latin and strip extra spaces if any
df_barkodovi['Oblik'] = df_barkodovi['Oblik'].apply(lambda x: cyr_to_lat(x).strip() if pd.notna(x) else x)
# Also capitalize Oblik to match the format in oblici.csv
df_barkodovi['Oblik'] = df_barkodovi['Oblik'].str.capitalize()

df_barkodovi['Doza_jedinica_mjere'] = df_barkodovi['Doza_jedinica_mjere'].apply(lambda x: x.strip() if isinstance(x, str) else x)

jedinice_mjere = pd.read_csv('jedinice_mjere.csv')
oblici = pd.read_csv('oblici.csv')

# Rename the index column from the CSVs for clarity
jedinice_mjere = jedinice_mjere.rename(columns={'Index': 'Doza_jedinica_mjere_index'})
oblici = oblici.rename(columns={'Index': 'Oblik_index'})

# Merge df_barkodovi with jedinice_mjere on the Doza_jedinica_mjere column
df_barkodovi = df_barkodovi.merge(jedinice_mjere[['Doza_jedinica_mjere', 'Doza_jedinica_mjere_index']], on='Doza_jedinica_mjere', how='left')

# Merge df_barkodovi with oblici on the Oblik column
df_barkodovi = df_barkodovi.merge(oblici[['Oblik', 'Oblik_index']], on='Oblik', how='left')

# Drop the original text-based Doza_jedinica_mjere and Oblik columns
df_barkodovi = df_barkodovi.drop(columns=['Doza_jedinica_mjere', 'Oblik'])

# Rename the merged columns to the desired names
df_barkodovi = df_barkodovi.rename(columns={'Doza_jedinica_mjere_index': 'Doza_jedinica_mjere', 'Oblik_index': 'Oblik'})

# Convert the index columns to strings (fill missing values with an empty string)
df_barkodovi['Doza_jedinica_mjere'] = df_barkodovi['Doza_jedinica_mjere'].fillna('').astype(str)
df_barkodovi['Oblik'] = df_barkodovi['Oblik'].fillna('').astype(str)

# Save the resulting CSV
df_barkodovi.to_csv('barkodovi.csv', index=False, header=True)

In [394]:
import pandas as pd
import os

# CSV file paths
csv_dir = '/Users/matom/Desktop/batric/pillbie-data'
jedinice_mjere_path = os.path.join(csv_dir, 'jedinice_mjere.csv')
vrste_path = os.path.join(csv_dir, 'vrste.csv')
barkodovi_path = os.path.join(csv_dir, 'barkodovi.csv')
output_sql_path = os.path.join(csv_dir, 'import_data.sql')

# Read CSV files
df_jedinice = pd.read_csv(jedinice_mjere_path)
df_vrste = pd.read_csv(vrste_path)
df_barkodovi = pd.read_csv(barkodovi_path)

# Show example of duplicate EANs
duplicate_eans = df_barkodovi[df_barkodovi.duplicated(subset=['EAN'], keep=False)]['EAN'].unique()
print(f"Number of EANs with duplicates: {len(duplicate_eans)}")
if len(duplicate_eans) > 0:
    print("Example of records with duplicate EANs:")
    print(df_barkodovi[df_barkodovi['EAN'] == duplicate_eans[0]].head())

# Remove duplicates - keeping only the first occurrence of each EAN
df_jedinice = df_jedinice.drop_duplicates(subset=['Doza_jedinica_mjere'])
df_vrste = df_vrste.drop_duplicates(subset=['Vrsta'])
# For barcodes, only keep the first occurrence of each EAN
df_barkodovi = df_barkodovi.drop_duplicates(subset=['EAN'], keep='first')

# Create SQL file
with open(output_sql_path, 'w') as sql_file:
    # Write header
    sql_file.write("-- Auto-generated SQL import script\n")
    sql_file.write("-- Generated from CSV files in pillbie-data (with duplicates removed)\n\n")
    
    # Optional: Add truncate statements (commented out by default)
    sql_file.write("-- Uncomment to clear existing data\n")
    sql_file.write("-- TRUNCATE TABLE medication_unit RESTART IDENTITY CASCADE;\n")
    sql_file.write("-- TRUNCATE TABLE medication_type RESTART IDENTITY CASCADE;\n")
    sql_file.write("-- TRUNCATE TABLE barcodes RESTART IDENTITY CASCADE;\n\n")
    
    # 1. Import medication units
    sql_file.write("-- Import medication units\n")
    for _, row in df_jedinice.iterrows():
        sql_file.write(f"INSERT INTO medication_unit (id, name) VALUES ({row['Index']}, '{row['Doza_jedinica_mjere']}');\n")
    
    sql_file.write("\n-- Reset the sequence\n")
    sql_file.write("SELECT setval('medication_unit_id_seq', (SELECT MAX(id) FROM medication_unit));\n\n")
    
    # 2. Import medication types
    sql_file.write("-- Import medication types\n")
    for _, row in df_vrste.iterrows():
        # Escape single quotes in names
        name = row['Vrsta'].replace("'", "''")
        sql_file.write(f"INSERT INTO medication_type (id, name) VALUES ({row['Index']}, '{name}');\n")
    
    sql_file.write("\n-- Reset the sequence\n")
    sql_file.write("SELECT setval('medication_type_id_seq', (SELECT MAX(id) FROM medication_type));\n\n")
    
    # 3. Import barcodes
    sql_file.write("-- Import barcodes\n")
    for _, row in df_barkodovi.iterrows():
        # Handle NULL values and escape single quotes in names
        name = str(row['Naziv']).replace("'", "''") if pd.notna(row['Naziv']) else ''
        strength = row['Doza_snaga'] if pd.notna(row['Doza_snaga']) else 'NULL'
        unit_id = row['Doza_jedinica_mjere'] if pd.notna(row['Doza_jedinica_mjere']) and row['Doza_jedinica_mjere'] != '' else 'NULL'
        type_id = row['Oblik'] if pd.notna(row['Oblik']) and row['Oblik'] != '' else 'NULL'
        
        sql_file.write(f"INSERT INTO barcodes (id, barcode, name, unit_id, type_id, strength) VALUES " +
                      f"({row['ID']}, '{row['EAN']}', '{name}', {unit_id}, {type_id}, {strength});\n")
    
    sql_file.write("\n-- Reset the sequence\n")
    sql_file.write("SELECT setval('barcodes_id_seq', (SELECT MAX(id) FROM barcodes));\n")

print(f"SQL insert script generated successfully at: {output_sql_path}")
print(f"Original records in jedinice_mjere: {len(pd.read_csv(jedinice_mjere_path))}, After removing duplicates: {len(df_jedinice)}")
print(f"Original records in vrste: {len(pd.read_csv(vrste_path))}, After removing duplicates: {len(df_vrste)}")
print(f"Original records in barkodovi: {len(pd.read_csv(barkodovi_path))}, After removing duplicates: {len(df_barkodovi)}")

Number of EANs with duplicates: 92
Example of records with duplicate EANs:
        ID            EAN       Naziv  Doza_snaga  Doza_jedinica_mjere  Oblik
0   394112  8606100000000  Mirapexin®        0.25                    8     54
1   394124  8606100000000  Mirapexin®        1.00                    8     54
15  397215  8606100000000    Topamax®       25.00                    8      2
16  397217  8606100000000    Topamax®       50.00                    8      2
17  397219  8606100000000    Topamax®      100.00                    8      2
SQL insert script generated successfully at: /Users/matom/Desktop/batric/pillbie-data/import_data.sql
Original records in jedinice_mjere: 8, After removing duplicates: 8
Original records in vrste: 5, After removing duplicates: 5
Original records in barkodovi: 3329, After removing duplicates: 110
